In [0]:
import json
from pyspark.sql import functions as f

In [0]:
api_key = dbutils.get()


In [0]:
# Widgets for job parametrization
spark.conf.set("spark.sql.session.timeZone", "UTC")

dbx = dbutils.widgets

dbx.text("csv_path", "/FileStore/tables/sales.csv")
dbx.text("csv_table", "sales_data_bronze")

dbx.text("repo_owner", "Yadu9238")
dbx.text("repo_name", "DataDesk")

print("Config Loaded ✅")

In [0]:


import requests
import pandas as pd
api_key = dbutils.secrets.get(scope="api_key", key="stock")
headers = {"X-Api-Key": f"{api_key}"}
url = f"https://stock.indianapi.in/NSE_most_active"

resp = requests.get(url, headers=headers, timeout=10)
resp.raise_for_status()
data = resp.json()
df = pd.json_normalize(data)
# Flatten JSON → Spark DF
api_df = spark.createDataFrame(df)
(
    api_df
    .write
    .mode("overwrite")
    .format("delta")
    .option("mergeSchema", "true")
    .saveAsTable("nse_table")
)
print("NSE Active Stock data created✅")




In [0]:
display(api_df)